In [0]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt



%matplotlib inline


Using TensorFlow backend.


In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['Dataset 2.zip']), 'r')
data.extractall()

In [0]:
data.printdir()

In [0]:
train_path = 'Dataset 2/train'
valid_path = 'Dataset 2/valid'
test_path = 'Dataset 2/test'

In [0]:
train_batches = ImageDataGenerator().flow_from_directory(
    train_path, target_size=(72,96), classes=['c0', 'c1','c2','c3','c4'], batch_size= 50)   
valid_batches = ImageDataGenerator().flow_from_directory(
    valid_path, target_size=(72,96), classes=['c0', 'c1','c2','c3','c4'], batch_size= 50) 
test_batches =  ImageDataGenerator().flow_from_directory(
    test_path, target_size=(72,96), classes=['c0', 'c1','c2','c3','c4'], batch_size= 50) 

Found 3200 images belonging to 2 classes.
Found 525 images belonging to 2 classes.
Found 550 images belonging to 2 classes.


In [0]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('0ff')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
        
        
        
        
        
        
        
            
        

In [0]:

imgs, labels = next(train_batches)


In [0]:
model = Sequential([
        Conv2D(32, (3 ,3), activation='relu', input_shape=(480, 640, 3)),
        Flatten(),
        Dense(2, activation='softmax')
        ])

In [0]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(train_batches, steps_per_epoch=32, validation_data=valid_batches, 
                    validation_steps= 21, epochs=5, verbose=2)

Epoch 1/5
 - 5s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 2/5
 - 3s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 3/5
 - 3s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 4/5
 - 3s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 5/5
 - 3s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619


In [0]:
test_imgs, test_labels = next(test_batches)



In [0]:
test_labels = test_labels[:,0]
test_labels

array([0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0.],
      dtype=float32)

In [0]:
predictions = model.predict_generator(test_batches, steps=11, verbose=0)

In [0]:
predictions

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [0]:
vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet', 
                                             include_top=False, input_shape=(480, 640, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
vgg16_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 24, 128)       0         
__________

In [0]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model


In [0]:

model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 24, 128)       0         
__________

In [0]:
output_vgg16_conv = vgg16_model(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)


In [0]:
model = Model(inputs=input, outputs=x)

In [0]:
model.summary()









_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 72, 96, 3)         0         
_________________________________________________________________
vgg16 (Model)                (None, 2, 3, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              12587008  
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 2)                 8194      
Total params: 44,091,202
Trainable params: 44,091,202
Non-trainable params: 0
________________________________________________________________

In [0]:
for layer in model.layers:
  layer.trainable = False

In [0]:
model.add(Dense(5, activation='softmax'))

NameError: ignored

In [0]:
model.summary()

In [0]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(train_batches, steps_per_epoch=32, validation_data=valid_batches, 
                    validation_steps= 21, epochs=20, verbose=2)
print('hello dasheed')





Epoch 1/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 2/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 3/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 4/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 5/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 6/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 7/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 8/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 9/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 10/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 11/20
 - 19s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_acc: 0.7619
Epoch 12/20
 - 20s - loss: 6.0443 - acc: 0.6250 - val_loss: 3.8376 - val_a

In [0]:
test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)

In [0]:
test_labels= test_labels[:,0]
test_labels

In [0]:
predictions = model.predict_generator(test_batches, steps=11, verbose=0)

In [0]:
cm = confusion_matrix(test_labels, np.round(predictions[:,0]))

In [0]:
cm_plot_labels = ['cat', 'dog']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')